
# TP 1 : Theorie des Graphes 1 
#### L3 INFO NEC 2024–2025 <br> Université de Pau et des Pays de l’Adour
###### date: "2024-12-18"

## prérequis

### installer avec pip (sur la machine)

c'est pareil avec Windows, Linux, Macos

##### version de python :

>si vous avez la version python ou python3 il suffit d'ajoute le 3 en fonction de votre version<br>
>et heuresement python fonctionne avec pip et python3 fonctionne avec pip3 (normalement les versions<br>
>actuelles installées sont pip3 avec python3, python n'est plus utilisé donc pip non plus)<br>

#### installer dash_cytoscape avec 2 packages
```sh
pip3 install dash
pip3 install dash-cytoscape
```

#### installer pip3 :

```sh
python3 -m ensurepip --upgrade
```
enlever 3 dans python3 pour les version ultérieures<br>
si problèmes voir : https://pip.pypa.io/en/stable/installation/



## <span style="color:#dd4444">avertissement</span> 

pour la partie graphique il vaut mieux lancer a nouveau tout le code depuis le début<br>
si on veut récupérer la sortie d'un algorithme entécédent car les données créées plus<br>
loin dans le code va altérer des données des graphes ultérieurs. Toutes les sorties graphiques<br>
faites avec __Cytoscape__ et __Dash__ vont être synchronisées sur les sorties de tout le notebook<br>
dans lequel les graphes sont faits.



### importation dans le code

In [16]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan

### on crée nos classes pour créer une structure de graphe

In [17]:
class Vertex:
    """gère les sommets et ses arrêtes (sous forme d'autres sommets)"""

    id = 0 # identifiant unique de tous les vertex dans les graphes
    
    def __init__(self, name, weight=0):
        """name:le nom du sommet (vertex) associé au noeud
        weight: le poids du sommet pour aller du noeud jusq'au sommet (par défaut=0)
        (nom du noeud non représenté ici, se produit dans la classe Node plus loin)"""
        self.name = name
        self.weight = weight # du sommet du noeud jusq'a ce sommet (ici)
        self.id = Vertex.id
        Vertex.id += 1
    
    def __str__(self):
        '''si on fait print(Vertex) le resultat du print est dans ce return'''
        return self.name

In [18]:
class Node:
    """
    représente les noeuds entre des point dans un graphe
    
    on crée la classe qui s'occupe de tous les noeuds entre 
    les sommets (vertex/vertices) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant unique de noeud auquel il appartient    

    def __init__(self, node_vertex:Vertex, vertices:list[Vertex]):
        self.name = node_vertex.name
        node_vertex.weight = 0 # le poids du sommet du noeud ne peux pas être différent de 0
        self.weight = sum([v.weight for v in vertices]) # poids du noeud
        self.vertices = vertices
        self.vertices_names = [v.name for v in self.vertices]
        self.degree = len(vertices)
        self.id = Node.id
        Node.id += 1

    def __str__(self, print_weight:bool=True):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.vertices)
        for i, e in enumerate(self.vertices):
            res += f"{e.name}"
            if(print_weight):
                res += f" w={e.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res += "," 
                res+=" "
        res+="]"
        return res
    


In [19]:
class Edge:
    '''représente une arrête entre deux sommets/vertex/noeud'''

    id = 0

    def __init__(self, vfrom:str | int, vto:str | int, weight):
        self.a = vfrom
        self.b = vto
        self.weight = weight
        self.id = Edge.id
        Edge.id += 1
    
    def __eq__(self, edge:object):
        '''compare un edge a un autre objet edge'''
        return self.id == edge.id
    
    def __ne__(self, edge:object):
        return not self.__eq__(edge)

In [39]:

class Graph:
    """créée un graphe avec une liste de "Node" en paramètre et un titre"""

    def __init__(self, data:list[Node], title:str="default"):
        self.title = title # titre du graphique si utilisé
        self.adj = data # liste d'adjacences
        self.edges = []
        self.nodesn = [] # liste des noeuds (seulement les noms)
        self.weight = 0 # poids total du graphe    
        self.degree = 0 # pas encore calculé le degré du graphe
        for n in self.adj:
            self.nodesn.append(n.name)
            self.weight += n.weight
            n.weight = None # le vertex/sommet de Node n'a jamais de poids
            for v in n.vertices:
                self.edges.append(Edge(n.name, v.name, v.weight))
                self.degree += 1
        self.create_matrix() # matrice des liens entre les noeuds (Edges) (matrice d'adjacences)

    def create_matrix(self):
        """créer la matrice d'ajdacence des points
        (fonctionne avec les graphes orienté également)
        """
        self.matrix = [[False for _ in self.adj] for _ in self.adj]
        self.is_oriented=False
        for lin, node in enumerate(self.adj): # parcours noms des noeuds
            for v in node.vertices: # parcours des "edges"/sommets/vertex
                self.matrix[lin][self.nodesn.index(v.name)]=True
                # si on trouve que m[i][j] != m[j][i] c'est oriente
                if(not self.is_oriented and node.name not in self.adj[self.nodesn.index(v.name)].vertices_names):
                    self.is_oriented=True

    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affichage au plus simple du graphe avec des caractères"""
        res = ""
        for i, node in enumerate(self.adj):
            res += f"{i}\t | {node.name} ["
            v_size = len(node.vertices)
            for j in range(v_size):
                res += f"{node.vertices[j].name}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def _sort_edges(self, edges:list[Edge])->list[Edge]: # complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances"""
        if len(edges)<2 :
            return edges
        else:
            pivot = edges[len(edges)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for e in edges:
                if(e.weight < pivot): l.append(e)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(e.weight == pivot): m.append(e) 
                else: r.append(e)
            return self._sort_edges(l)+m+self._sort_edges(r)

    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        """
        self.edges = self._sort_edges(self.edges)

    def show_edges(self):
        print([e.a+e.b+" w="+str(e.weight) for e in self.edges])
    
    def render(self, layoutname="breadthfirst"):
        """effectue le rendu du graphe visuellement"""
        unique = " "+f'{time()%1e3:.5}'
        app = Dash(self.title+unique)
        allow_arrows = "linear" # ce style n'autorise pas les flèches
        if self.is_oriented:
            allow_arrows = "bezier" # ce style oui
        custom_layout = {
            'width': '100%', 
            'height': '500px', 
            "border": "3px white solid",
            "border-radius":"5px",
            "background-color":"#666666",
            "title" : {"background-color":"white"}
        }
        styles = [{
                'selector': 'node',  
                'style': {
                    'background-color': '#222222', 
                    'color': 'white',            
                    'label': 'data(label)',       
                    'font-size': '16px',          
                    'text-valign': 'center',     
                    'text-halign': 'center'       
                }
            },
            {
                'selector': 'edge', 
                'style': {
                    'width': 2, 
                    'target-arrow-shape': "vee", 
                    "target-arrow-color": "#4a7cf2",
                    'arrow-scale': 2,
                    'curve-style': allow_arrows
                }
            }
        ]
        elems = [] # éléments à afficher (formattés)
        for node in self.adj:
            elems.append({'data': {"id":node.name, "label":node.name}})
        # add edges
        for edge in self.edges:
            elems.append({'data': {'source': edge.a, 'target': edge.b}})
        app.layout = html.Div([
            cyto.Cytoscape(
                id='cytoscape'+unique,
                elements=elems,
                layout={'name': layoutname},
                style=custom_layout,
                stylesheet=styles
            )
        ])
        print("également ouvert sur la page web : \"localhost:8050\"")     
        print("opened too at the web page : \"localhost:8050\"")   
        app.run_server(debug=True,port=8051)


In [40]:
# Tests :

# tester l'efficacité des calculs avec les maths

def generate_unoriented(size = 26):
    f"""génère un graphe non orienté sans boucles (sur un même noeud) de {size} noeuds"""
    # stocakge des adjacences pour compléter les Node dans la liste de Node "nodes"
    res = None
    if(size > 1 and size < 27):
        vertices = [Vertex(chr(i+65),0) for i in range(size)]
        adj = [[] for _ in range(size)] 
        for i in range(vertices):
            for _ in range(randint(0,size)): # nb added
                pos = randint(0,25) # index added
                pos += 1 if pos == i else 0
                rand_vertex = Vertex(chr(pos+65),randint(1,400))
                adj[i].append(rand_vertex)
                adj[pos].append(vertices[i])
        res = Graph([Node(vertices[i], adj[i]) for i in range(size)])
    else:
        raise Exception(f"not enought or too many vertices to generate unoriented graph (1 < n={size} < 27)")
    return res


# fonction temporaire pour générer une liste de sommets aléatoire (avec edges aléatoire)
# sans avoir de boucle sur le même noeud (paramètre v) exemple: 'v'->'v'
# ORIENTE
tmpadata2 = lambda v : [Vertex(chr(i+65), randint(1,400)) for i in range(randint(1,26)) if i != v]
data2 = [Node(Vertex(chr(j+65)), tmpadata2(j)) for j in range(26)]
graphe2 = Graph(data2)

print(graphe2)
graphe2.show_edges()
graphe2.sort_by_weight()
graphe2.show_edges()



0	 | A [B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S]
1	 | B [A, C, D, E, F, G, H, I, J, K, L, M, N, O, P]
2	 | C [A, B, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W]
3	 | D [A, B, C, E, F, G, H, I, J, K, L, M, N]
4	 | E [A, B, C, D, F, G, H, I, J, K, L, M, N, O, P]
5	 | F [A, B, C, D, E, G, H, I, J, K, L, M, N, O]
6	 | G [A, B, C, D, E, F, H, I, J, K, L, M, N, O, P, Q, R]
7	 | H [A, B, C, D, E, F, G, I]
8	 | I [A, B, C, D, E, F]
9	 | J [A, B, C, D, E, F, G, H, I, K, L, M, N, O, P, Q, R, S, T, U]
10	 | K [A, B]
11	 | L [A, B, C, D, E, F, G, H, I, J, K, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z]
12	 | M [A, B, C, D, E, F, G, H, I, J, K, L, N, O, P, Q, R, S, T, U, V, W, X, Y, Z]
13	 | N [A, B, C, D, E, F, G, H, I, J, K, L, M, O, P, Q, R, S, T, U, V]
14	 | O [A, B, C, D, E, F, G, H, I, J, K, L, M, N, P, Q, R, S, T]
15	 | P [A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, Q, R, S, T, U, V, W, X, Y]
16	 | Q [A, B, C, D, E, F, G, H, I]
17	 | R [A, B, C, D, E, F]
18	 | S [A,

In [41]:
# breadthfirst(default) grid circle concentric cose random preset
graphe2.render("grid")

également ouvert sur la page web : "localhost:8050"
opened too at the web page : "localhost:8050"


## saisie des données

#### données fournies :

> 1: 2(2), 3(1)<br>
> 2: 1(2), 4(2), 5(3)<br>
> 3: 1(1), 2(3), 4(2)<br>
> 4: 2(2), 3(5), 5(2), 6(4)<br>
> 5: 2(3), 4(2), 6(2)<br>
> 6: 4(2), 5(2)<br>

il y a effectivement une erreur (3->2) sur les données d'origine (il manque donc (2->3))<br>
donc on rajoute et ça donne ça :

> 1: 2(2), 3(1)<br>
> 2: 1(2), 4(2), 5(3), 3(3)<br>
> 3: 1(1), 2(3), 4(2)<br>
> 4: 2(2), 3(5), 5(2), 6(4)<br>
> 5: 2(3), 4(2), 6(2)<br>
> 6: 4(2), 5(2)<br>


In [42]:
# rappel format du noeud : 
# Noeud(nomActuel, [Vertex("nomLié1", poid1),Vertex("nomLié2", poid2),etc...])
data = [
    Node(Vertex("1"),[Vertex("2",2),Vertex("3",1)]),
    Node(Vertex("2"),[Vertex("1",2),Vertex("4",2),Vertex("5",3),Vertex("3",3)]),
    Node(Vertex("3"),[Vertex("1",1),Vertex("2",3),Vertex("4",2)]),
    Node(Vertex("4"),[Vertex("2",2),Vertex("3",5),Vertex("5",2),Vertex("6",4)]),
    Node(Vertex("5"),[Vertex("2",3),Vertex("4",2),Vertex("6",2)]),
    Node(Vertex("6"),[Vertex("4",2),Vertex("5",2)])
]
graphe = Graph(data,"exemple de graphe")
print(np.matrix(graphe.matrix))


# on vérifie que tout est juste
ok = [[False, True, True, False, False, False],
      [True, False, True, True, True, False],
      [True, True, False, True, False, False],
      [False, True, True, False, True, True],
      [False, True, False, True, False, True],
      [False, False, False, True, True, False]]
assert ok == graphe.matrix, "une des valeurs n'est pas vraie"

[[False  True  True False False False]
 [ True False  True  True  True False]
 [ True  True False  True False False]
 [False  True  True False  True  True]
 [False  True False  True False  True]
 [False False False  True  True False]]


## Kruskal

> init: arrêtes d'ordre ascendant de poids (croissant)

on peut utiliser `sort()` qui existe a la fois dans python et dans RStudio<br>
> pour i=1...n-1 des sommets<br>
> &emsp;prendre l'arrête de poids min qui ne fait pas une boucle<br>
> &emsp;et qui n'est pas déja dans la liste des arrêtes que l'on a déja choisit<br>
> fin

### remarque

on a besoin de DFS ou un algorithme avancé que l'on a pas vu en cours<br>
pour savoir si on a une boucle ou non !!!


In [45]:
# on fait de l'héritage car on peut pas 
# rajouter la méthode kruskal comme en swift avec des extensions
class GraphK(Graph):
    """rajoute l'algorithme de Kruskal et un algorithme pour trouver des cycles"""

    # obligatoire (heritage)
    def __init__(self, data, title = "default"):
        super().__init__(data, title)

    # obligatoire
    def __init__(self, gr:Graph):
        """convertit un Graph normal en GraphK"""
        super().__init__(gr.adj,gr.title)

    @staticmethod
    def dfs(graph:Graph)->list[str]:
        '''Fonction DFS pour trouver le chemin le plus court entre 2 sommets
        Depth First Search (algorithme de parcours en profondeur)'''
        visited = set(graph.adj[0].name)
        path = list(graph.adj[0].name)

        if(graph.is_oriented):
            # code here
            pass
        else:
            # code here
            pass

        return path


    @staticmethod
    def get_cycles(edges: list[Edge]) -> list[list[str]]:
        '''Détecte les cycles dans un graphe orienté ou non orienté
        attention cependant sur les graphe orienté ça peut prendre beaucoup de temps'''
        visited = set()  # Pour marquer les sommets visités
        path = []  # suivre le chemin actuel
        cycles = []  # stocker les cycles trouvés
        data_node = {} # dictionnaire des noms des sommets associé a la liste d'adjacence des autre sommets
        weights = {} # poids des adjacences des sommets de data_node
        
        # Construire la représentation du graphe à partir des objets Edge
        for edge in edges:
            if edge.a not in data_node: # compare les clefs de data_node uniquement
                data_node[edge.a] = []
                weights[edge.a] = []
            if edge.b not in data_node:
                data_node[edge.b] = []
                weights[edge.b] = []
            data_node[edge.a].append(edge.b)
            weights[edge.a].append(edge.weight)
        # construction du graphe avec les données
        graph = GraphK(
            [Node(Vertex(n), [Vertex(v, weights[v]) for v in vx]) for n, vx in data_node if not len(vx)]
        )

        if(graph.is_oriented):
            # code a mettre ici
            # 1 cycle = minimum 2 sommets et 2 arrêtes de sens opposé entre ces sommets
            pass
        else:
            # 1 cycle = minimum 3 sommets et qu'ils soient tous connectés entre eux
            if(len(graph.adj)<3):
                raise Exception(f"nombre de sommets insuffisants ({len(graph.adj)} < 3)")
            tmp = list()
            carry = True
            current = graph.adj[0].name
            visited.add(graph.adj[0].name)
            while(carry):
                # code a mettre ici

                # a partir du 3e element on va pouvoir faire des cycles donc la boucle doit changer 
                # de comportement a ce moment la
                
                if(len(visited) == len(graph.adj)):
                    carry=False # déja tout visité
                    
                    


        return cycles


    def kruskal(self,red_tarjan_rule=False)->list[Edge]:
        """renvoie l'arbre couvrant de poids minimal
        applique la règle rouge de tarjan si booléen est True (False par défaut)"""
        # comme vue en cours
        # trie les arrêtes de tout le graphe par poids croissants
        self.sort_by_weight()
        if(len(self.adj)<3):
            raise Exception(f"under minimum required data ({len(self.adj)} vertices < 3)")
        # copie des éléments
        tmp_edges = list(self.edges) 
        tmin = list(tmp_edges.pop(0))
        carry = True # stopper la boucle
        idx = 0
        for edge in self.edges: # pour chaque arretes
            # 1er arrete qui n'est pas dans tmin (forcément la min) et qui fait pas de cycle
            while(carry and idx < len(tmin)): # parcours de tmin
                if(tmin[idx] != edge and not len(GraphK.get_cycles(tmin))):
                    tmin.append(edge)
                idx+=1
            # reset
            carry=True
            idx = 0



        

In [46]:
# vérification (données insérées a la main pour être sûr)
graphe = GraphK(graphe) # conversion
print(graphe)
#graphe.sort_weight_ASC()
#print(graphe)
graphe.render()
print(graphe.has_cycle(graphe.edges))

0	 | 1 [2, 3]
1	 | 2 [1, 4, 5, 3]
2	 | 3 [1, 2, 4]
3	 | 4 [2, 3, 5, 6]
4	 | 5 [2, 4, 6]
5	 | 6 [4, 5]

également ouvert sur la page web : "localhost:8050"
opened too at the web page : "localhost:8050"


[]


methode kruskall en R : 
```R
kruskal <- function(sommets,arretes,poids){
    poids_ord <- sort(poids,index.return=true)
    poids <- poids_ord$x
    index_poids <- poids_ord$ix 
    return poids
}
```